In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
import pandas as pd
from sklearn.model_selection import StratifiedKFold

In [2]:
import py_entitymatching as em
A = em.read_csv_metadata('new1_1.csv', key='ID')
B = em.read_csv_metadata('new2_1.csv', key='ID')


No handlers could be found for logger "py_entitymatching.io.parsers"


In [ ]:
ob = em.OverlapBlocker()
C1 = ob.block_tables(A, B, 'Name', 'Name', word_level=True, overlap_size=3, 
                    l_output_attrs=list(A.columns), 
                    r_output_attrs=list(B.columns),
                    show_progress=False)
C3 = ob.block_candset(C1, 'Resolution', 'Resolution',overlap_size=4,show_progress=False)
len(C3)

830

In [ ]:
l_data = em.read_csv_metadata('labeled_data1.csv', key='_id', 
                         fk_ltable='ltable_ID', fk_rtable='rtable_ID',
                         ltable=A, rtable=B)
match_f = em.get_features_for_matching(A, B)
H = em.extract_feature_vecs(l_data, feature_table=match_f,attrs_before = None, attrs_after=['Class'])
H.fillna(value=0, inplace=True)

The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,ID,ID,numeric,numeric,Exact Match; Absolute Norm
1,3.5mm jack,3.5mm jack,short string (1 word to 5 words),short string (1 word),Not Applicable: Types do not match
2,Battery,Battery,numeric,numeric,Exact Match; Absolute Norm
3,Clock Speed,Clock Speed,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
4,Card slot,Card slot,medium string (5 words to 10 words),short string (1 word to 5 words),Not Applicable: Types do not match
5,GPS Support,GPS Support,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
6,GPU,GPU,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
7,Internal Storage,Internal Storage,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
8,NFC,NFC,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
9,Name,Name,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"


In [ ]:
train_test = em.split_train_test(H, train_proportion=0.7)
train_set = train_test['train']
test_set = train_test['test']
train_set.to_csv('training_split')
test_set.to_csv('test_split')
train_labels = train_set['Class']
test_labels = test_set['Class']
#del train_set['Class']

In [ ]:
rf= em.RFMatcher()
rf.fit(table=train_set, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'Class'], target_attr='Class')
match_f = em.get_features_for_matching(A, B)
new = em.extract_feature_vecs(C3, feature_table=match_f,attrs_before = None)
new.fillna(value=0, inplace=True)


op = rf.predict(table=new,  exclude_attrs=['_id', 'ltable_ID', 'rtable_ID'],  append=True, target_attr='Predicted_Labels')


df1=op[op['Predicted_Labels']==1]
df2=df1[['ltable_ID','rtable_ID']]

df3=op[op['Predicted_Labels']==0]
unmatched_id=df3[['ltable_ID','rtable_ID']]


In [ ]:
df2


In [ ]:

x = A[A['ID'].isin(df2['ltable_ID'])]
y = B[B['ID'].isin(df2['rtable_ID'])]


In [ ]:
x.head()

In [ ]:
y.reset_index(drop=True)

In [ ]:
dfn = pd.concat([x,y], axis=1)
dfn.to_csv("matches.csv")

In [ ]:
l_list1 = [93, 104, 272, 296, 361, 462, 492, 602, 705, 850, 865, 915, 920, 1014, 1084, 1106, 1136, 1184, 1326, 1392, 1474, 1511, 1561, 1625, 1775, 1926, 1932, 2038, 2072, 2108, 2124, 2139, 2150, 2167, 2234, 2241, 2292, 2333, 2390, 2410, 2470, 2534, 2591, 2594, 2600, 2666, 2680, 2695, 2710, 2754, 2831, 2885, 3025, 3127, 3175, 3224]

In [ ]:
l_list2 = list(unmatched_id['ltable_ID'])
rlist = list(unmatched_id['rtable_ID'])
set_l = set(l_list1) | set(l_list2)
set_r = set(rlist)

final_l_list = list(set_l)
final_rlist = list(set_r)

In [ ]:
A.head()

In [ ]:
A[A['ID'] == 93]

In [ ]:
C3[C3['ltable_ID']==93]['ltable_Name']

In [ ]:
filtered_l = A[A['ID'].isin(final_l_list)]
filtered_r = B[B['ID'].isin(final_rlist)]
df = filtered_l.append(filtered_r)
df


In [ ]:
len(final_l_list)

In [ ]:
len(final_rlist)

In [ ]:
rlist2 = [1, 21, 22, 24, 60, 77, 80, 81, 123, 133, 134, 162, 165, 171, 179, 231, 239, 243, 343, 349, 355, 358, 486, 504, 548, 568, 629, 636, 665, 749, 755, 830, 861, 921, 1006, 1010, 1066, 1077, 1078, 1158, 1163, 1398, 1404, 1407, 1418, 1473, 1488, 1493]

In [ ]:
set_r2 = set(rlist2)

In [ ]:
set_r_inter = set_r & set_r2

In [ ]:
final_rlist = list(set_r - set_r_inter)

In [ ]:
filtered_l = A[A['ID'].isin(final_l_list)]
filtered_r = B[B['ID'].isin(final_rlist)]
df = filtered_l.append(filtered_r)
df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
#%matplotlib inline
data=pd.read_csv("Merged2.csv")
sec_cam=data['Secondary Camera'].apply(lambda x:str(x).split("MP")[0])
data.insert(1,"sec_cam",sec_cam)





In [ ]:
import py_entitymatching as em
A = em.read_csv_metadata('new1_1.csv', key='ID')#Files before cleaning
B = em.read_csv_metadata('new2_1.csv', key='ID')


rf= em.RFMatcher()
rf.fit(table=train_set, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'Class'], target_attr='Class')
match_f = em.get_features_for_matching(A, B)
new = em.extract_feature_vecs(C3, feature_table=match_f,attrs_before = None)
new.fillna(value=0, inplace=True)


op = rf.predict(table=new,  exclude_attrs=['_id', 'ltable_ID', 'rtable_ID'],  append=True, target_attr='Predicted_Labels')


df1=op[op['Predicted_Labels']==1]
df1.to_csv("matches.csv")
df2=df1[['ltable_ID','rtable_ID']]
df2.to_csv("matching.csv")
df3=op[op['Predicted_Labels']==0]

unmatched_id=df3[['ltable_ID','rtable_ID']]


l_lmatched = set([93, 104, 272, 296, 361, 462, 492, 602, 705, 850, 865, 915, 920, 1014, 1084, 1106, 1136, 1184, 1326, 1392, 1474, 1511, 1561, 1625, 1775, 1926, 1932, 2038, 2072, 2108, 2124, 2139, 2150, 2167, 2234, 2241, 2292, 2333, 2390, 2410, 2470, 2534, 2591, 2594, 2600, 2666, 2680, 2695, 2710, 2754, 2831, 2885, 3025, 3127, 3175, 3224])
r_matched = set([93, 104, 272, 296, 361, 462, 492, 602, 705, 850, 865, 915, 920, 1014, 1084, 1106, 1136, 1184, 1326, 1392, 1474, 1511, 1561, 1625, 1775, 1926, 1932, 2038, 2072, 2108, 2124, 2139, 2150, 2167, 2234, 2241, 2292, 2333, 2390, 2410, 2470, 2534, 2591, 2594, 2600, 2666, 2680, 2695, 2710, 2754, 2831, 2885, 3025, 3127, 3175, 3224])
l_unmatched = set(list(unmatched_id['ltable_ID']))
r_unmatched = set(list(unmatched_id['rtable_ID']))
op1= l_lmatched.union(l_unmatched)
op2= r_unmatched.difference(r_matched) 
out1 = A[A['ID'].isin(op1)]
out2 = B[B['ID'].isin(op2)]
datanew = out1.append(out2)
datanew.to_csv("final_list.csv")